In [1]:
import pandas as pd
data = pd.read_csv(r"C:\GEE project\Singapore\MYD11A2.061_1km_aid0001.csv")

In [2]:
data

,Unnamed: 0,time,lat,lon,crs,LST_Day_1km,LST_Night_1km,QC_Day,QC_Night
0,0,27-12-2021 00:00,1.554167,103.495833,-127,301.92,NaN,65.0,2.0
1,1,27-12-2021 00:00,1.554167,103.504167,-127,301.88,NaN,65.0,2.0
2,2,27-12-2021 00:00,1.554167,103.512500,-127,301.44,NaN,65.0,2.0
3,3,27-12-2021 00:00,1.554167,103.520833,-127,299.74,NaN,65.0,2.0
4,4,27-12-2021 00:00,1.554167,103.529167,-127,298.48,NaN,65.0,2.0
...,...,...,...,...,...,...,...,...,...
159983,159983,01-01-2023 00:00,1.179167,104.070833,-127,NaN,NaN,3.0,3.0
159984,159984,01-01-2023 00:00,1.179167,104.079167,-127,305.16,NaN,NaN,2.0
159985,159985,01-01-2023 00:00,1.179167,104.087500,-127,305.74,NaN,NaN,2.0
159986,159986,01-01-2023 00:00,1.179167,104.095833,-127,305.22,NaN,65.0,2.0


In [4]:
data.dropna(subset=['LST_Day_1km'], inplace=True)


In [5]:
data

,Unnamed: 0,time,lat,lon,crs,LST_Day_1km,LST_Night_1km,QC_Day,QC_Night
0,0,27-12-2021 00:00,1.554167,103.495833,-127,301.92,NaN,65.0,2.0
1,1,27-12-2021 00:00,1.554167,103.504167,-127,301.88,NaN,65.0,2.0
2,2,27-12-2021 00:00,1.554167,103.512500,-127,301.44,NaN,65.0,2.0
3,3,27-12-2021 00:00,1.554167,103.520833,-127,299.74,NaN,65.0,2.0
4,4,27-12-2021 00:00,1.554167,103.529167,-127,298.48,NaN,65.0,2.0
...,...,...,...,...,...,...,...,...,...
159977,159977,01-01-2023 00:00,1.179167,104.020833,-127,308.56,NaN,65.0,2.0
159984,159984,01-01-2023 00:00,1.179167,104.079167,-127,305.16,NaN,NaN,2.0
159985,159985,01-01-2023 00:00,1.179167,104.087500,-127,305.74,NaN,NaN,2.0
159986,159986,01-01-2023 00:00,1.179167,104.095833,-127,305.22,NaN,65.0,2.0


In [6]:
import numpy as np
#import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from shapely.geometry import Point


# Load your historical stock data
# Replace 'your_data.csv' with the path to your data file
#data = pd.read_csv(r"C:\GEE project\Singapore\MYD11A2.061_1km_aid0001.csv")

# Preprocess the data
data['time'] = pd.to_datetime(data['time'])
data.set_index('time', inplace=True)

# Normalize the data
scaler = MinMaxScaler()
# Combine latitude and longitude to create a geometry column with Point objects
geometry = [Point(lon, lat) for lon, lat in zip(data['lon'], data['lat'])]
gdf = gpd.GeoDataFrame(data, geometry=geometry, crs='EPSG:4326')

C:\Users\arpan\AppData\Local\Temp\ipykernel_7452\2863036064.py:16: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data['time'] = pd.to_datetime(data['time'])


In [7]:
data=gdf
# Create sequences for the LSTM
sequence_length = 20  # Tweak this value based on your data
sequences = []
next_data_point = []

for i in range(len(data) - sequence_length):
    sequences.append(data[['geometry','LST_Day_1km']].values[i:i + sequence_length])
    next_data_point.append(data[['geometry', 'LST_Day_1km']].values[i + sequence_length])

X = np.array(sequences)
y = np.array(next_data_point)

MemoryError: Unable to allocate 427. KiB for an array with shape (2, 27321) and data type object